In [49]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy.io import arff
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.spatial.distance import pdist, cdist
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.utils import check_random_state
from scipy.spatial import distance
from tslearn.soft_dtw_fast import *
from tslearn.cysax import *
from tslearn.metrics import dtw, dtw_path,dtw_path_from_metric,soft_dtw
from tslearn.utils import *


# Loading Training and Testing Data Source

In [50]:
data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TRAIN.arff')
data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TRAIN.arff')
data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TRAIN.arff')

Test_data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TEST.arff')
Test_data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TEST.arff')
Test_data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TEST.arff')

In [51]:
df_D1 = pd.DataFrame(data1[0])
df_D2 = pd.DataFrame(data2[0])
df_D3 = pd.DataFrame(data3[0])

Test_df_D1 = pd.DataFrame(Test_data1[0])
Test_df_D2 = pd.DataFrame(Test_data2[0])
Test_df_D3 = pd.DataFrame(Test_data3[0])

## Make Over the data, as desired

In [52]:
df_D1 =df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
df_D2 =df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
df_D3 =df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

Test_df_D1 =Test_df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
Test_df_D2 =Test_df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
Test_df_D3 =Test_df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

In [53]:
df_D1 = df_D1.drop('target', axis=1)
df_D2 = df_D2.drop('target', axis=1)
df_D3 = df_D3.drop('target', axis=1)

Test_df_D1 = Test_df_D1.drop('target', axis=1)
Test_df_D2 = Test_df_D2.drop('target', axis=1)
Test_df_D3 = Test_df_D3.drop('target', axis=1)

In [54]:
Sort_Dim_1 = np.expand_dims(df_D1.values.reshape(-1),axis=1)
Sort_Dim_2 = np.expand_dims(df_D2.values.reshape(-1),axis=1)
Sort_Dim_3 = np.expand_dims(df_D3.values.reshape(-1),axis=1)

Sort_Test_Dim_1 = np.expand_dims(Test_df_D1.values.reshape(-1),axis=1)
Sort_Test_Dim_2 = np.expand_dims(Test_df_D2.values.reshape(-1),axis=1)
Sort_Test_Dim_3 = np.expand_dims(Test_df_D3.values.reshape(-1),axis=1)

In [55]:
Training_View = np.concatenate((Sort_Dim_1, Sort_Dim_2),axis=1)
Training_View = np.concatenate((Training_View, Sort_Dim_3),axis=1)

In [56]:
Training_View.shape

(258804, 3)

In [57]:
Testing_View = np.concatenate((Sort_Test_Dim_1, Sort_Test_Dim_2),axis=1)
Testing_View = np.concatenate((Testing_View, Sort_Test_Dim_3),axis=1)

In [58]:
finalarray_test_data = Testing_View[:][0:182]

In [59]:
finalarray_test_data.shape

(182, 3)

## Initializing the data

In [96]:
Total_rows = 258804
Sample_Percentage = 0
Number_of_run= 0
Length_RTS = 0
global Sub_Sample_copy_PTS 
Training_Results_List = []
PTS_Results_List = []
Store_Avg_list = []
dictionary = dict()
RTS_short_list_Results_List = []
Starting_point = Testing_View[:][0:1]
End_point = Testing_View[:][181:182]

## Functions body for Random TS

In [61]:
def Calculate_Percentage(Sample_Percentage):
    Refining_Percentage= Total_rows * (Sample_Percentage/100)
    Refining_Percentage = round(Refining_Percentage)   
    return Refining_Percentage

In [62]:
def Random_Part(Refining_Percentage,Length_RTS):
    Sub_Sampling = Training_View[:][0:Refining_Percentage]

    number_of_rows = Sub_Sampling.shape[0]
    random_indices = np.random.choice(number_of_rows, size=Length_RTS, replace=False)
    random_rows = Sub_Sampling[random_indices, :]
    return random_rows
    

In [63]:
def Adding_start_end_point(random_rows):
    Output_TS = np.concatenate((Starting_point, random_rows),axis=0)
    Output_TS = np.concatenate((Output_TS, End_point),axis=0)
    return Output_TS

In [64]:
def Calculate_Random_DTW(Output_TS,Len_RTS):
    path, dist = dtw_path(finalarray_test_data, Output_TS)
    if(dist>0 and Len_RTS >100 ):
        
        Training_Results_List.append(dist)  
    else:
        RTS_short_list_Results_List.append(dist)

## Functions body for Pseudo TS

In [65]:
def generate_similarity_matrix(Complete_PTS):
    ED_Matrix = distance.cdist(Complete_PTS, Complete_PTS, 'euclidean')
    return ED_Matrix

In [66]:
def generate_graph(similarity_output):
    G = nx.from_numpy_matrix(similarity_output, create_using=nx.Graph)
    layout = nx.spring_layout(G)
    sizes = len(similarity_output)
    return G

In [67]:
def generate_MST(G):
    layout = nx.spring_layout(G)
    T=nx.minimum_spanning_tree(G)
    return T

In [68]:
def generate_Dijkstra(T):
    Dij = nx.dijkstra_path(T,0,181)
    return Dij

In [69]:
def doing_miscellaneous_work(Dijkstra_output):
    a = []
    for x in range(len(Dijkstra_output)):
    
        Index = Dijkstra_output[x]
        a.append(list(Complete_PTS[Index]))
    P_TS = np.array(a)
    return P_TS

In [70]:
def generate_DTW_PTS(Miscellaneous_output):
    path, dist = dtw_path(finalarray_test_data, Miscellaneous_output)
    if(dist>0):
        PTS_Results_List.append(dist)     

In [71]:
def Adding_start_end_point_PTS(Pseudo_TS):
    Output_PTS = np.concatenate((Starting_point, Pseudo_TS),axis=0)
    Output_PTS = np.concatenate((Output_PTS, End_point),axis=0)
    return Output_PTS

In [72]:
def Run_all_fun(Complete_PTS):
    similarity_output = generate_similarity_matrix(Complete_PTS)
    graph_output = generate_graph(similarity_output)
    MST_output_1 =  generate_MST(graph_output)
    Dijkstra_output = generate_Dijkstra(MST_output_1)
    Miscellaneous_output = doing_miscellaneous_work(Dijkstra_output)
    generate_DTW_PTS(Miscellaneous_output)

### Running Part

In [73]:
for x in range(1,6):
    Refining_Percentage = Calculate_Percentage(20*x)
    random_rows = Random_Part(Refining_Percentage,180)
    Output_TS = Adding_start_end_point(random_rows)
    Calculate_Random_DTW(Output_TS,180)
print(Training_Results_List)    

[23.81115825269829, 22.532286078529157, 24.555857829419796, 23.96471349720488, 24.257705429146572]


In [74]:
for x in range(1,6):
    Refining_Percentage = Calculate_Percentage(20*x)
    random_rows = Random_Part(Refining_Percentage,70)
    Output_TS = Adding_start_end_point(random_rows)
    Calculate_Random_DTW(Output_TS,70)
print(RTS_short_list_Results_List)



[14.555088703248117, 16.264208953635805, 15.894743298217437, 14.749562599398498, 15.14613263904892]


In [97]:
for i in range (10):
    for x in range(1,6):
        Refining_Percentage = Calculate_Percentage(20*x)
        random_rows = Random_Part(Refining_Percentage,70)
        Output_TS = Adding_start_end_point(random_rows)
        Calculate_Random_DTW(Output_TS,70)
    dictionary[i] = RTS_short_list_Results_List
    RTS_short_list_Results_List= []

In [98]:
dictionary

{0: [15.43565178677227,
  16.262969053219344,
  15.90447339534148,
  13.686210104755006,
  15.649819684552538],
 1: [15.702125912755312,
  16.816079042290564,
  15.206226750548112,
  15.37770545981701,
  16.279040047186324],
 2: [14.798573037045836,
  16.775775562704933,
  14.659725086276348,
  14.197336891048367,
  15.29371578750769],
 3: [14.430958894868317,
  15.360345097175221,
  15.837868527239387,
  15.604880494965382,
  15.369686925519076],
 4: [14.29412950622426,
  15.678891097251936,
  15.351111788841683,
  15.546489841050454,
  13.69357316046685],
 5: [15.367670486098865,
  15.190510473560034,
  15.46672420041626,
  16.745225780938938,
  14.961590369863664],
 6: [16.10101165549084,
  15.078860868422453,
  15.915448689006412,
  16.07781107004843,
  14.029438570836865],
 7: [15.546590913749805,
  15.185186381836742,
  15.662825897038024,
  15.213739749776154,
  13.962021059729501],
 8: [13.85976407249997,
  14.891315046678992,
  15.682473519431687,
  16.95661921232375,
  15.774

In [87]:
Store_Avg_list 

15.61564901776847

In [80]:
RTS_short_list_Results_List

149.0142090875086

###  ------------------------------------------

In [214]:
for x in range(1,6):
    i = 181
    if(x==1):
        
        Sub_Sampling_PTS = Training_View[:][1:181*x]
        Pseudo_TS = Sub_Sampling_PTS
        Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS)
        Run_all_fun(Complete_PTS)  
    else:
        
        Sub_Sampling_PTS = Training_View[:][1+i:181*x]
        Pseudo_TS = Sub_Sampling_PTS
        Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS)
        Run_all_fun(Complete_PTS) 
 

In [215]:
print(PTS_Results_List)

[10.05884167027412, 9.3343652050839, 9.78243724869677, 9.797855373367529, 9.80363631840538]
